In [1]:
import os
import time
from stable_baselines3 import PPO
from StarCraft2Env import StarCraft2Env
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv,SubprocVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.env_util import make_vec_env

In [2]:
model_name = f'{int(time.time())}'
model_dir = f'models/{model_name}/'
logs_dir = f'logs/{model_name}/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

In [3]:
def make_env(seed = 0):
    def _init():
        env = StarCraft2Env(seed)
        env.seed(seed)
        return env
    return _init
# monitor_dir = r'./monitor_log/'
# os.makedirs(monitor_dir,exist_ok=True)
# env = Monitor(env,monitor_dir)
num_cpu = 12
env = SubprocVecEnv([make_env(i) for i in range(num_cpu)])
# env = DummyVecEnv([lambda: env])
# env = make_vec_env(env,n_envs=4,monitor_dir=monitor_dir,vec_env_cls=SubprocVecEnv)

In [4]:
model = PPO("CnnPolicy", env, verbose=1,tensorboard_log=logs_dir,device='cuda',batch_size=128,learning_rate=1e-3)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [5]:
TIMESTEPS = 50000
iters = 0
while True:
    iters += 1
    model.learn(total_timesteps=TIMESTEPS,tb_log_name='PPO',reset_num_timesteps=False)
    model.save(f"{model_dir}/{TIMESTEPS*iters}")

Logging to logs/1661663814/PPO_0
------------------------------
| time/              |       |
|    fps             | 27    |
|    iterations      | 1     |
|    time_elapsed    | 448   |
|    total_timesteps | 12288 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 23          |
|    iterations           | 2           |
|    time_elapsed         | 1051        |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.014266455 |
|    clip_fraction        | 0.0603      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.94       |
|    explained_variance   | 0.0017      |
|    learning_rate        | 0.001       |
|    loss                 | -0.037      |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0172     |
|    value_loss           | 0.0212      |
------------------------------------


KeyboardInterrupt



In [ ]:
del model # remove to demonstrate saving and loading
iters = 0
model = PPO.load(f"{model_dir}/{TIMESTEPS*iters}")

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()
